Praktikum NLP:

ToDo:
- Load dataset √

- Dependency Parsing

- Extracting aspects (phrases/sentiment-words)

    - Summarize aspects for each hotel

    - Calculate sentiment scores for each aspect (TextBlob, VADER, etc...)

- Analyse 5 most common aspects

- Dump into graphs for the presentation

In [83]:
#Load data

import pandas as pd
import re
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
import string
from collections import Counter
from textblob import TextBlob

#nltk.download('wordnet')
#nltk.download('omw-1.4')

with open('../../include/Hotel_Reviews.csv') as file:
    df = pd.read_csv(file, nrows=10000)

nlp = spacy.load("en_core_web_sm")




In [84]:
## Clean data

# Delete redundant and empty reviews

def preprocessing_reviews(df):
    df.drop(df[ df['Negative_Review'] == 'No Negative'].index)
    df.drop(df[ df['Positive_Review'] == 'No Positive'].index)
    df.drop_duplicates(keep=False)

# Split into sentences and add dots to positive reviews and negative reviews
    for text in df['Positive_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
            print(sentence)

        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t|n\'t|nt', ' not',complete_review)
        df['Positive_Review'] = df['Positive_Review'].replace([text], complete_review)

    for text in df['Negative_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t|n\'t|nt', ' not',complete_review)
        df['Negative_Review'] = df['Negative_Review'].replace([text], complete_review)

    return df




df = preprocessing_reviews(df)
pd.set_option("display.max_colwidth", None) # -1?

#TODO: Maybe i and I will lead to problems...
#TODO: payments <= payme nots, restaurants <= restaura not, romantic <= Roma notic
# My room was dirty and. I was afraid to walk barefoot o nothe floor 

# Quick check if wanted
#print(df['Negative_Review'])
#print(df['Positive_Review'])




# Remove stopwords

#nlp = spacy.load("en_core_web_sm")
#stopword_list = nltk.corpus.stopwords.words('english')


# lower case


# Lemmatization

#print(df)




Only the park outside of the hotel was beautiful 
No real complaints the hotel was great great location surroundings rooms amenities and service 
Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one 
Bit confusing 
Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great 
Aside from those minor minor things fantastic spot and will be back when i return to 
Amsterdam 
Location was good and staff were ok 
It is cute hotel the breakfast range is nice 
Will go back 
Great location in nice surroundings the bar and restaurant are nice and have a lovely outdoor area 
The building also has quite some character 
Amazing location and building 
Romantic setting 
Good restaurant with modern design great chi

In [85]:
aspects_pos = []
aspects_neg = []


for review in df["Negative_Review"]:
  doc = nlp(review)
  descriptors = ''
  aspect = ''
  for token in doc:
    negation = False
    for negative_token in doc:
      if negative_token.dep_ == 'neg' and negative_token.head == token: negation = True 
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      aspect = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV': 
          continue
        prepend += child.text + ' '
      descriptors = prepend + token.text
  aspects_neg.append({'aspect': aspect,
    'description': descriptors, 'negation': negation})
aspects_neg = pd.DataFrame(aspects_neg)
print(pd.DataFrame(aspects_neg))

  #negation.extend([tok for tok in doc if tok.dep_ == 'neg'])
for review in df["Positive_Review"]:
  doc = nlp(review)
  descriptors = ''
  aspect = ''
  for token in doc:
    negation = False
    for negative_token in doc:
      if negative_token.dep_ == 'neg' and negative_token.head == token: negation = True 
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      aspect = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptors = prepend + token.text
  aspects_pos.append({'aspect': aspect,
    'description': descriptors, 'negation': negation})
aspects_pos = pd.DataFrame(aspects_pos)
print(pd.DataFrame(aspects_pos))





      aspect   description  negation
0       room         newer     False
1                 Negative     False
2      rooms         empty     False
3      offer         ready     False
4     photos         happy     False
...      ...           ...       ...
9995              Negative     False
9996                           False
9997          Very cramped     False
9998              Negative     False
9999                           False

[10000 rows x 3 columns]
        aspect      description  negation
0         park        beautiful     False
1       option         notastic     False
2        range             nice     False
3     building          outdoor     False
4                         notic     False
...        ...              ...       ...
9995                                False
9996     staff  really friendly     False
9997                   Friendly     False
9998      Beds          helpful     False
9999                    horible     False

[10000 rows x 3 columns]


In [86]:

for boolean in aspects_neg['negation']:
    if boolean == True: print("true")

for boolean in aspects_pos['negation']:
    if boolean == True: print("true")


true


In [87]:
"""countStemmer = Counter()

for aspect in aspects['aspect']:
    aspect = stemmer.stem( aspect)
    aspect = aspect.lower()
    countStemmer[aspect] += 1

print(countStemmer)"""


lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

countLemmaPos = Counter()
countLemmaNeg = Counter()


for aspect in aspects_pos['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaPos[aspect] += 1


for aspect in aspects_neg['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaNeg[aspect] += 1

print(countLemmaNeg, "\n")
print(countLemmaPos)



    

Counter({'': 5923, 'room': 668, 'staff': 198, 'hotel': 167, 'bathroom': 160, 'bed': 121, 'breakfast': 108, 'rooms': 78, 'shower': 63, 'area': 53, 'service': 46, 'location': 45, 'view': 45, 'people': 42, 'restaura': 41, 'window': 39, 'price': 38, 'bar': 37, 'nothey': 36, 'door': 35, 'food': 33, 'water': 31, 'size': 30, 'conditioning': 30, 'disappoi': 29, 'bedroom': 29, 'floor': 24, 'reception': 21, 'one': 21, 'thing': 19, 'pillow': 18, 'system': 17, 'manager': 16, 'noise': 14, 'parking': 14, 'stay': 14, 'heating': 13, 'mattress': 13, 'lift': 13, 'facility': 13, 'place': 12, 'cost': 12, 'receptionist': 12, 'guest': 11, 'member': 11, 'egg': 11, 'table': 11, 'experience': 11, 'wall': 10, 'coffee': 10, 'work': 10, 'building': 10, 'bath': 10, 'space': 10, 'toilet': 10, 'tap': 10, 'rest': 10, 'person': 10, 'unit': 10, 'screen': 10, 'night': 9, 'choice': 9, 'wifi': 9, 'temperature': 9, 'issue': 9, 'nothere': 8, 'fridge': 8, 'tea': 8, 'housekeeping': 8, 'menu': 8, 'curtain': 8, 'husband': 8, 't

In [88]:
sentiment_pos = []
for aspect in aspects_pos['description']:
  sentiment_pos.append(TextBlob(aspect).sentiment)
aspects_pos['sentiment'] = sentiment_pos

sentiment_neg = []
for aspect in aspects_neg['description']:
  sentiment_neg.append(TextBlob(aspect).sentiment)
aspects_neg['sentiment'] = sentiment_neg


In [89]:
print(aspects_neg)

      aspect   description  negation    sentiment
0       room         newer     False   (0.0, 0.0)
1                 Negative     False  (-0.3, 0.4)
2      rooms         empty     False  (-0.1, 0.5)
3      offer         ready     False   (0.2, 0.5)
4     photos         happy     False   (0.8, 1.0)
...      ...           ...       ...          ...
9995              Negative     False  (-0.3, 0.4)
9996                           False   (0.0, 0.0)
9997          Very cramped     False   (0.2, 0.3)
9998              Negative     False  (-0.3, 0.4)
9999                           False   (0.0, 0.0)

[10000 rows x 4 columns]


In [90]:
# Dependency parsing

import nltk



def dependency_parsing(df):
    for sentence in df:
        break
  #      doc = nlp(sentence)




In [91]:
# In Addition:

# I assume we are not supposed to do topic modeling, just "simple" rule based aspect extraction... right?

# make Sentiment Analysis (only simple one: Textblob) for positive and negative reviews and check, if it fits,
#                otherwise drop the review
# -> example: "I am so angry that i made this post available via all possible sites i use when planing my trips so
#              no one will make the mistake of booking this place"
# maybe handle orthographical mistakes before splitting into sentences
#



In [92]:
# Define rules for extracting aspects: keywords:

# Aspect food: restaurant, breakfast, dinner, lunch, bar
# Aspect helpfulness: staff, personell, clerk, service, check-in
# Aspect price: cheap, expensive, euros, ...?
# Aspect cleanliness: bed, matress, bathroom, floor, sheets, "changed daily", shower
# Aspect features/equipment:  USB-port, wifi, airconditioning, showergel, shampoo, furniture, roomservice, bathtub
# Aspect connectivity: bus, tram, train, airportshuttle, parking
# Aspect property: stairs (steep), building, (under-)construction, renovation, location, citycenter, sightseeing


Presentation: Spatial-Mapping where hotels are // where guests are from
=> maybe what aspects are mentioned by primarily by guests by nationality or positive/negative in general

